In [1187]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [1188]:
df1 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\pcb_2020.csv')
df2 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\pcb_2021.csv')


In [1189]:
file_path = r'D:\Seksatta\MKG\summary lizard R&D\pcb_2022.csv'
df3 = pd.read_csv(file_path, encoding='utf-8')

In [1190]:
df = pd.concat([df1,df2, df3], 
          keys=[2020, 2021, 2022])

In [1191]:
df.reset_index(inplace =True)
df.drop(columns = ['level_1'], axis=1, inplace=True)

In [1192]:
df.rename(columns={'level_0':'year'}, inplace=True)

In [1193]:
df['error_code'] = df['Problem'].str.extract(r'(\b\w{2})\s')

In [1194]:
def antmal(x):
    ant_match = re.search(r'\b[Aa]nt\b(?!\s*\band\b)', x)
    lizard_match = re.search(r'\b[Ll][ie](?!\s*\band\b)', x)
    
    if ant_match and lizard_match:
        return 'both'
    elif ant_match:
        return 'ant'
    elif lizard_match:
        return 'lizard'
    else:
        return 'none'



In [1195]:
df['animal'] = df['Visual check'].apply(antmal)

In [1196]:
df.columns

Index(['year', 'MKG No.', 'Parts  and cover page', 'Finished date', 'Period',
       'Part', 'Model', 'Serial', 'Drawing', 'Problem', 'Vender name',
       'Visual check', 'Resitance check', 'Operation check by Jig', 'Judgment',
       '  MKG No.', 'Parts  received from MKG',
       'Finished basic investigation date', 'Send parts to supplier date',
       'Part Name', 'Model Type', 'Resistance check', 'error_code', 'animal'],
      dtype='object')

In [1197]:
df2 = df.loc[:,['year', 'MKG No.', 'Model', 'Serial', 'Drawing', 'error_code', 'animal']]
df2

,year,MKG No.,Model,Serial,Drawing,error_code,animal
0,2020,2210.0,2MKC30QVM4,E007339,2P438792-2,NaN,ant
1,2020,75.0,RKQ12TV2S,E007399,2P416384-10,NaN,lizard
2,2020,162.0,FTKQ15TV2S,E006048,2P406592-11,NaN,lizard
3,2020,204.0,FTKQ09TV2S,E002809,2P409592-11,A6,ant
4,2020,144.0,ARKC12RV2S,E009028,2P390290-1,L5,lizard
...,...,...,...,...,...,...,...
228,2022,NaN,RKZ12VV2S,E002061,2P582816-24,NaN,lizard
229,2022,NaN,RKQ15UV2S,E001163,2P582816-1,L5,lizard
230,2022,NaN,RKQ24UV2S,E007651,2P494704-13,E7,lizard
231,2022,NaN,FTKQ15UV2S,E003301,2P406592-11,NaN,lizard


In [1198]:
df.groupby(['year','animal'])['Serial'].count()

year  animal
2020  ant       16
      both       3
      lizard    84
2021  both       5
      lizard    93
2022  lizard    32
Name: Serial, dtype: int64

In [1199]:
model = pd.read_csv(r'D:\Seksatta\EQG\dit_info\model2.csv')
model['model_name'] = model['model_name'].str.strip()

In [1200]:
model.columns

Index(['Unnamed: 0', 'model_name', 'i_o', 'base_model', 'group'], dtype='object')

**ข้อมูลจิ้งจก เก็บอยู่ใน df3**

In [1201]:
df2.columns

Index(['year', 'MKG No.', 'Model', 'Serial', 'Drawing', 'error_code',
       'animal'],
      dtype='object')

In [1203]:
df3 = pd.merge(df2, model[['model_name', 'base_model','group', 'i_o']], left_on='Model', right_on='model_name', how='left')


In [1207]:
df3

,year,MKG No.,Model,Serial,Drawing,error_code,animal,model_name,base_model,group,i_o
0,2020,2210.0,2MKC30QVM4,E007339,2P438792-2,NaN,ant,2MKC30QVM4,BMS,RA,o
1,2020,75.0,RKQ12TV2S,E007399,2P416384-10,NaN,lizard,RKQ12TV2S,GSI,RA,o
2,2020,162.0,FTKQ15TV2S,E006048,2P406592-11,NaN,lizard,FTKQ15TV2S,BMS,RA,i
3,2020,204.0,FTKQ09TV2S,E002809,2P409592-11,A6,ant,FTKQ09TV2S,BMS,RA,i
4,2020,144.0,ARKC12RV2S,E009028,2P390290-1,L5,lizard,ARKC12RV2S,GSI,RA,o
...,...,...,...,...,...,...,...,...,...,...,...
228,2022,NaN,RKZ12VV2S,E002061,2P582816-24,NaN,lizard,RKZ12VV2S,BMS,RA,o
229,2022,NaN,RKQ15UV2S,E001163,2P582816-1,L5,lizard,RKQ15UV2S,BMS,RA,o
230,2022,NaN,RKQ24UV2S,E007651,2P494704-13,E7,lizard,RKQ24UV2S,SF2,RA,o
231,2022,NaN,FTKQ15UV2S,E003301,2P406592-11,NaN,lizard,FTKQ15UV2S,BMS,RA,i


In [1209]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        233 non-null    int64  
 1   MKG No.     201 non-null    float64
 2   Model       233 non-null    object 
 3   Serial      233 non-null    object 
 4   Drawing     233 non-null    object 
 5   error_code  104 non-null    object 
 6   animal      233 non-null    object 
 7   model_name  229 non-null    object 
 8   base_model  229 non-null    object 
 9   group       229 non-null    object 
 10  i_o         229 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 20.2+ KB


In [1220]:
df4 = df3.groupby(['year','group', 'i_o','animal'])['Serial'].size().reset_index()
df4.rename(columns = {'Serial':'count'}, inplace=True)

In [1222]:
df4['i_o'] = df4.i_o.map({'i':'indoor', 
            'o':'outdoor'})

In [1219]:
df4.to_excel(r'pcb_lizard_ant.xlsx')